In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('UNSW_NB15_testing-set.csv')

data.drop('proto', axis=1, inplace=True)
data.drop('service', axis=1, inplace=True)
data.drop('label', axis=1, inplace=True)

# Using pd.get_dummies() on the 'attack_cat' column
dummies = pd.get_dummies(data['attack_cat'], dtype=int)

# Concatenate the dummies DataFrame with the original DataFrame
data_with_dummies = pd.concat([data, dummies], axis=1)

data_with_dummies.drop('attack_cat', axis=1, inplace=True)


array_data = data_with_dummies.values

data_with_dummies.info()

data_with_dummies

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175341 entries, 0 to 175340
Data columns (total 25 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   dur               175341 non-null  float64
 1   spkts             175341 non-null  int64  
 2   dpkts             175341 non-null  int64  
 3   sbytes            175341 non-null  int64  
 4   dbytes            175341 non-null  int64  
 5   rate              175341 non-null  float64
 6   sttl              175341 non-null  int64  
 7   dttl              175341 non-null  int64  
 8   sload             175341 non-null  float64
 9   dload             175341 non-null  float64
 10  ct_srv_src        175341 non-null  int64  
 11  ct_dst_ltm        175341 non-null  int64  
 12  ct_src_dport_ltm  175341 non-null  int64  
 13  ct_dst_sport_ltm  175341 non-null  int64  
 14  ct_dst_src_ltm    175341 non-null  int64  
 15  Analysis          175341 non-null  int32  
 16  Backdoor          17

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,Analysis,Backdoor,DoS,Exploits,Fuzzers,Generic,Normal,Reconnaissance,Shellcode,Worms
0,0.121478,6,4,258,172,74.087490,252,254,1.415894e+04,8495.365234,...,0,0,0,0,0,0,1,0,0,0
1,0.649902,14,38,734,42014,78.473372,62,252,8.395112e+03,503571.312500,...,0,0,0,0,0,0,1,0,0,0
2,1.623129,8,16,364,13186,14.170161,62,252,1.572272e+03,60929.230470,...,0,0,0,0,0,0,1,0,0,0
3,1.681642,12,12,628,770,13.677108,62,252,2.740179e+03,3358.622070,...,0,0,0,0,0,0,1,0,0,0
4,0.449454,10,6,534,268,33.373826,254,252,8.561499e+03,3987.059814,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,0.000009,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,...,0,0,0,0,0,1,0,0,0,0
175337,0.505762,10,8,620,354,33.612649,254,252,8.826286e+03,4903.492188,...,0,0,0,0,0,0,0,0,1,0
175338,0.000009,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,...,0,0,0,0,0,1,0,0,0,0
175339,0.000009,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,...,0,0,0,0,0,1,0,0,0,0


In [13]:
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from sklearn.preprocessing import StandardScaler
from keras.layers import Dropout
from keras.regularizers import l2

actual_input = array_data[:,0:15]
actual_output = array_data[:,15:25]

scaler = StandardScaler()
# min_max_scaler = preprocessing.MinMaxScaler()
scale_input = scaler.fit_transform(actual_input)

X_train, X_test, y_train, y_test = train_test_split(scale_input, actual_output, test_size=0.2, random_state=42)

# # Step 3: Build the DNN model
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(64, activation='relu', input_shape=(15,)),  # Input layer with 64 neurons and ReLU activation
#     tf.keras.layers.Dense(32, activation='relu'),  # Hidden layer with 32 neurons and ReLU activation
#     tf.keras.layers.Dense(11, activation='sigmoid')  # Output layer with 1 neuron and Sigmoid activation (binary classification)
# ])

# # Step 4: Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Step 5: Train the model
# epochs = 100
# batch_size = 13800

# model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, activation="relu", input_shape=(15,)))
model.add(tf.keras.layers.Dense(32, activation="relu"))  
model.add(tf.keras.layers.Dense(10, activation="softmax"))
model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])

# # Use Adam optimizer with a lower learning rate
# optimizer = Adam(learning_rate=0.002)

# model.compile(loss="binary_crossentropy",
#                    optimizer=optimizer,
#                    metrics=["accuracy"])

model.fit(X_train, y_train, epochs=100,
                         batch_size=128,  # Experiment with different batch sizes
                         validation_data=(X_test, y_test)
                         )


# Step 6: Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")



Epoch 1/100
1096/1096 [==============================] - 2s 1ms/step - loss: 0.1503 - accuracy: 0.6705 - val_loss: 0.1189 - val_accuracy: 0.7244
Epoch 2/100
1096/1096 [==============================] - 1s 950us/step - loss: 0.1140 - accuracy: 0.7404 - val_loss: 0.1116 - val_accuracy: 0.7483
Epoch 3/100
1096/1096 [==============================] - 1s 943us/step - loss: 0.1086 - accuracy: 0.7502 - val_loss: 0.1070 - val_accuracy: 0.7539
Epoch 4/100
1096/1096 [==============================] - 1s 944us/step - loss: 0.1051 - accuracy: 0.7553 - val_loss: 0.1041 - val_accuracy: 0.7568
Epoch 5/100
1096/1096 [==============================] - 1s 955us/step - loss: 0.1028 - accuracy: 0.7589 - val_loss: 0.1019 - val_accuracy: 0.7648
Epoch 6/100
1096/1096 [==============================] - 1s 943us/step - loss: 0.1010 - accuracy: 0.7621 - val_loss: 0.1004 - val_accuracy: 0.7662
Epoch 7/100
1096/1096 [==============================] - 1s 999us/step - loss: 0.0998 - accuracy: 0.7650 - val_loss: 0.0

In [14]:
model.save("model_file.h5")

C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tkinter as tk
import tensorflow as tf
from sklearn import preprocessing
import numpy as np

model = tf.keras.models.load_model("model_file.h5")

def predict():
    attack_text = ["Analysis","Backdoor","DoS","Exploits"",Fuzzer","Generic","Normal","Reconnaissance","Shellcode","Worms"]
    input_values = [[float(input1.get()), float(input2.get()), float(input3.get()), float(input4.get()), float(input5.get()), float(input6.get()), float(input7.get()), float(input8.get()), float(input9.get()), float(input10.get()), float(input11.get()), float(input12.get()), float(input13.get()), float(input14.get()), float(input15.get())]]
    #preprocessed_input = np.array([input_values])
    #min_max_scaler = preprocessing.MinMaxScaler()
    #scale_input = min_max_scaler.fit_transform(preprocessed_input)
    result = model.predict(input_values)
#     result_label = result[0][0]
    result_attack = [(result[0][0]),(result[0][1]),(result[0][2]), (result[0][3]), (result[0][4]), (result[0][5]), (result[0][6]), (result[0][7]), (result[0][8]), (result[0][9])]
#     result_attack2 = np.array(result_attack)
#     result_attack_index = find_nearest(result_attack2)
    
    result_attack_index = np.argmax(result[0][0:])
    
#     label=""
    
#     if result_label == 1.0:
#         label = "attack"
#     else:
#         label = "no attack"
        
        
#     output_label1.config(text=f"attack category: {result_label}")
    
    output_label2.config(text=f"label: {attack_text[result_attack_index]}")


# def find_nearest(instance):
        
#     min_difference = 10.00
#     k=0
#     for num in instance:
#         difference = abs(num-1.0)
#         if difference < min_difference:
#             min_difference = difference
#             index = k
#             k = k + 1
#     return index
    
root = tk.Tk()
root.title("ann")
root.geometry("600x600")

label1 = tk.Label(root, text="dur:")
label1.grid(row=0, column=0)
input1 = tk.Entry(root)
input1.grid(row=0, column=1)

label2 = tk.Label(root, text="spkts:")
label2.grid(row=1, column=0)
input2 = tk.Entry(root)
input2.grid(row=1, column=1)

label3 = tk.Label(root, text="dpkts:")
label3.grid(row=2, column=0)
input3 = tk.Entry(root)
input3.grid(row=2, column=1)

label4 = tk.Label(root, text="sbytes:")
label4.grid(row=3, column=0)
input4 = tk.Entry(root)
input4.grid(row=3, column=1)

label5 = tk.Label(root, text="dbytes:")
label5.grid(row=4, column=0)
input5 = tk.Entry(root)
input5.grid(row=4, column=1)

label6 = tk.Label(root, text="rate:")
label6.grid(row=5, column=0)
input6 = tk.Entry(root)
input6.grid(row=5, column=1)

label7 = tk.Label(root, text="sttl:")
label7.grid(row=6, column=0)
input7 = tk.Entry(root)
input7.grid(row=6, column=1)

label8 = tk.Label(root, text="dttl:")
label8.grid(row=7, column=0)
input8 = tk.Entry(root)
input8.grid(row=7, column=1)

label9 = tk.Label(root, text="sload:")
label9.grid(row=8, column=0)
input9 = tk.Entry(root)
input9.grid(row=8, column=1)

label10 = tk.Label(root, text="dload:")
label10.grid(row=9, column=0)
input10 = tk.Entry(root)
input10.grid(row=9, column=1)

label11 = tk.Label(root, text="ct_srv_src:")
label11.grid(row=10, column=0)
input11 = tk.Entry(root)
input11.grid(row=10, column=1)

label12 = tk.Label(root, text="ct_dst_ltm:")
label12.grid(row=11, column=0)
input12 = tk.Entry(root)
input12.grid(row=11, column=1)

label13 = tk.Label(root, text="ct_src_dport_ltm:")
label13.grid(row=12, column=0)
input13 = tk.Entry(root)
input13.grid(row=12, column=1)

label14 = tk.Label(root, text="ct_dst_sport_ltm:")
label14.grid(row=13, column=0)
input14 = tk.Entry(root)
input14.grid(row=13, column=1)

label15 = tk.Label(root, text="ct_dst_src_ltm:")
label15.grid(row=14, column=0)
input15 = tk.Entry(root)
input15.grid(row=14, column=1)

button = tk.Button(root, text = "predict", command=predict)
button.grid(row=15, column=1)

output_label1 = tk.Label(root, text="")
output_label1.grid(row=16, column=1)

output_label2 = tk.Label(root, text="")
output_label2.grid(row=17, column=1)

root.mainloop()